# Enhanced Environment Manager

## New Features Demonstration

This notebook demonstrates the new features added to the Environment Manager:

1. **YAML Cleanup Function** - Remove old exported environment files
2. **Jupyter Kernel Recreation** - Update kernel paths for environments
3. **Enhanced Interactive Menu** - New options for cleanup and kernel management

---

## 1. Import and Initialize Environment Manager

In [ ]:
# Import the enhanced environment manager
from environment_manager import EnvironmentManager
from pathlib import Path
import json

# Initialize the manager
manager = EnvironmentManager()
print("✅ Environment Manager initialized")
print(f"Export directory: {manager.export_dir}")
print(f"Backup directory: {manager.backup_dir}")

## 2. YAML Cleanup Function

The cleanup function removes old YAML files from the exported_environments directory.

In [ ]:
# Check current YAML files
yaml_files = list(manager.export_dir.glob("*.yml")) + list(manager.export_dir.glob("*.yaml"))
print(f"Current YAML files: {len(yaml_files)}")
for file in yaml_files[:5]:  # Show first 5
    print(f"  - {file.name}")
if len(yaml_files) > 5:
    print(f"  ... and {len(yaml_files) - 5} more")

In [ ]:
# Create a test YAML file for demonstration
test_yaml = manager.export_dir / "demo_test.yml"
test_yaml.write_text("""name: demo_test
channels:
  - defaults
  - conda-forge
dependencies:
  - python=3.9
  - numpy
  - pandas
""")

print(f"Created test YAML file: {test_yaml.name}")
print(f"File contents:")
print(test_yaml.read_text())

In [ ]:
# Test the cleanup function (without confirmation for demo)
print("Testing YAML cleanup function...")
success = manager.cleanup_exported_yaml_files(confirm=False)
print(f"\nCleanup result: {'✅ Success' if success else '❌ Failed'}")

# Verify cleanup
remaining_files = list(manager.export_dir.glob("*.yml")) + list(manager.export_dir.glob("*.yaml"))
print(f"Remaining YAML files: {len(remaining_files)}")

## 3. Environment Detection for Kernels

The manager can detect which environments have Python or R installed.

In [ ]:
# Get all environments
environments = manager.list_environments()
print(f"Found {len(environments)} environments:")

# Check which environments have Python or R
python_envs = []
r_envs = []

for env in environments:
    if env['name'] in ['base', 'root']:
        continue
        
    has_python = manager._environment_has_python(env['path'])
    has_r = manager._environment_has_r(env['path'])
    
    print(f"  {env['name']:15} - Python: {'✓' if has_python else '✗'}, R: {'✓' if has_r else '✗'}")
    
    if has_python:
        python_envs.append(env['name'])
    if has_r:
        r_envs.append(env['name'])

print(f"\nEnvironments with Python: {python_envs}")
print(f"Environments with R: {r_envs}")

## 4. Jupyter Kernel Recreation

The kernel recreation function creates or updates Jupyter kernels for environments.

In [ ]:
# Check current kernel directories
kernel_base_dir = Path.home() / ".local" / "share" / "jupyter" / "kernels"
print(f"Kernel base directory: {kernel_base_dir}")

if kernel_base_dir.exists():
    existing_kernels = list(kernel_base_dir.iterdir())
    print(f"Existing kernels: {len(existing_kernels)}")
    for kernel in existing_kernels:
        print(f"  - {kernel.name}")
else:
    print("Kernel directory does not exist yet")

In [ ]:
# Test kernel recreation for one environment
if python_envs:
    test_env = python_envs[0]
    print(f"Testing kernel recreation for environment: {test_env}")
    
    success = manager.recreate_jupyter_kernels([test_env])
    print(f"Kernel recreation result: {'✅ Success' if success else '❌ Failed'}")
    
    # Check the created kernel
    kernel_dir = manager._get_kernel_dir(test_env)
    kernel_json = kernel_dir / "kernel.json"
    
    if kernel_json.exists():
        print(f"\nKernel created at: {kernel_dir}")
        with open(kernel_json, 'r') as f:
            kernel_config = json.load(f)
        print("Kernel configuration:")
        print(json.dumps(kernel_config, indent=2))
else:
    print("No Python environments available for testing")

## 5. Interactive Menu Features

The enhanced interactive menu now includes:

1. Process all environments
2. Select specific environments  
3. Preview mode (show changes without processing)
4. **🆕 Clean up exported YAML files**
5. **🆕 Recreate Jupyter kernels**
6. Exit

### Usage Examples:

In [ ]:
# Example of using the manager programmatically
print("🔧 Programmatic Usage Examples:")
print()

# 1. Cleanup with confirmation
print("1. Cleanup YAML files:")
print("   manager.cleanup_exported_yaml_files(confirm=True)")

# 2. Recreate all kernels
print("\n2. Recreate all kernels:")
print("   manager.recreate_jupyter_kernels()")

# 3. Recreate specific kernels
print("\n3. Recreate specific kernels:")
print("   manager.recreate_jupyter_kernels(['env1', 'env2'])")

# 4. Interactive mode
print("\n4. Interactive mode:")
print("   manager.run_interactive_mode()")

## 6. Summary and Benefits

### New Features:

1. **YAML Cleanup Function**
   - Removes old exported environment files
   - Prevents directory clutter
   - Optional confirmation prompt
   - Comprehensive logging

2. **Jupyter Kernel Recreation**
   - Updates kernel paths for renamed/moved environments
   - Preserves existing kernel configurations
   - Supports both Python and R kernels
   - Works with all or selected environments

3. **Enhanced Interactive Menu**
   - New menu options for cleanup and kernel management
   - Selective environment processing
   - User-friendly interface with confirmations

### Benefits:

- **Automated Maintenance**: Easy cleanup of old files
- **Jupyter Integration**: Seamless kernel management
- **Flexibility**: Work with all or specific environments
- **Safety**: Confirmation prompts prevent accidental operations
- **Logging**: Complete audit trail of all operations

### Usage:

```bash
# Run the interactive manager
python environment_manager.py

# Then choose from the menu:
# Option 4: Clean up exported YAML files
# Option 5: Recreate Jupyter kernels
```